# ニューラルネットワーク

## パーセプトロンからニューラルネットワークへ

パーセプトロンを再び考える

![perceptron0](./img/perceptron0.png)

ここでバイアスについて、重みがbで値が1固定の入力信号であると考えると、以下のように変形できる

![perceptron_bias](./img/perceptron_bias.png)

これによって、パーセプトロンの式を $入力値×重み$ に統一することができる（$a = \sum_{i=1}^n x_i*w_i$）

さらに、出力値 $y$ を得るための処理として、パーセプトロンでは $a \leqq 0 → 0$ ｜ $a > 0 → 1$ という条件分岐を行っている

このように、入力信号の総和を出力信号に変換するための処理関数を**活性化関数**と呼び、$h(a)$ で表す

つまり、パーセプトロンにおける活性化関数は `h(a) = 0 if a <= 0 else 1` と表現できる

このように、特定条件で0から1へ突然出力値が変わるような活性化関数を**ステップ関数**と呼ぶ

![perceptron_h](./img/perceptron_h.png)

### シグモイド関数
ステップ関数は微分することができないため、ニューラルネットワークにおいては、ステップ関数の代わりにシグモイド関数がよく使われてきた

$$
    シグモイド関数: h(a) = \frac{1}{1 + e^{-a}}
$$

In [1]:
"""
ステップ関数とシグモイド関数の比較
"""

# ステップ関数
step(a::Float64)::Float64 = (a <= 0 ? 0 : 1)

# シグモイド関数
sigmoid(a::Float64)::Float64 = 1 / (1 + exp(-a))

# プロット用｜横軸
## collect(start:interval:end): start〜endまでinterval刻みの数値配列を生成
a = collect(-5.0:0.1:5.0)

101-element Array{Float64,1}:
 -5.0
 -4.9
 -4.8
 -4.7
 -4.6
 -4.5
 -4.4
 -4.3
 -4.2
 -4.1
 -4.0
 -3.9
 -3.8
  ⋮  
  3.9
  4.0
  4.1
  4.2
  4.3
  4.4
  4.5
  4.6
  4.7
  4.8
  4.9
  5.0

In [2]:
# ステップ関数実行
## function.(array) で function(v) for v in array を実行
y = step.(a)

101-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [3]:
# Plotsパッケージ追加
using Pkg
Pkg.add("Plots")

# Plotsパッケージ利用
using Plots
plotly() # Choose the Plotly.jl backend for web interactivity

# ステップ関数プロット
plot(a, y)

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


┌ Info: Recompiling stale cache file /home/user/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
# シグモイド関数プロット
plot(a, sigmoid.(a))

<!DOCTYPE html>
 
 
 Plots.jl

### ステップ関数とシグモイド関数
ステップ関数とシグモイド関数は、上記グラフを見ても分かるように、滑らかさ（連続性）に違いがある

一方で、以下のような共通点もある

- 出力値は $0 \leqq y \leqq 1$ となる
    - 入力値が小さいほど 0 に近く、入力値が大きいほど 1 に近くなる
- 非線形関数である
    - 一時直線では表現できない関数である

### 非線形関数
ニューラルネットワークでは、活性化関数に非線形関数を用いる必要がある

逆に言えば、活性化関数に線形関数を用いてはならない

これは、線形関数を用いてしまうと、どれだけ層を深くしても、それと同じことを行う「隠れ層（中間層）のないネットワーク」が存在することになってしまうためである

例えば、活性化関数として $h(x) = cx$ という線形関数を採用し、$y(x) = h(h(h(x)))$ という3層ネットワークを構築したとする

このネットワークは $y(x) = c * c * c * x$ という乗算処理を行うが、$c$ は任意定数であることから、$y(x) = ax$（$a = c^3$）という1層ネットワークに等しくなってしまう

このように、線形関数を活性化関数として採用してしまうと、層を深くする意味がなくなってしまうため、必ず非線形関数を活性化関数とする必要がある

### ReLU関数
シグモイド関数は古くから活性化関数としてよく使われてきたが、最近では**ReLU（Rectified Linear Unit）関数**が採用されることが増えてきた

ReLU関数は、入力値が0以下なら0を出力し、入力値が0を超えたらその値をそのまま出力する

そのため、出力値が収束することはなく、より複雑な問題を解くことが可能である

In [7]:
# ReLU関数
ReLU(x::Float64)::Float64 = (x <= 0 ? 0 : x)

# プロット用｜入力値: -5.0, -4.9, ..., 4.9, 5.0
x = collect(-5.0:0.1:5.0)

# ReLU関数プロット
plot(x, ReLU.(x))

<!DOCTYPE html>
 
 
 Plots.jl

## 3層ニューラルネットワーク

### 3層ニューラルネットワークの実装
以下のような構造の3層ニューラルネットワークを実装する

![neural_network_3](./img/neural_network_3.png)

重みの記述を $w_{n p}^{(i)}$（n: 次層のニューロンの番号, p: 前層のニューロンの番号, i: 層の番号）とし、バイアスを加えると以下のようなネットワーク構造となる

![neural_network_bias](./img/neural_network_bias.png)

さらに、各中間層（隠れ層）に活性化関数 $h()$ を導入し、層同士の信号伝達を表現すると以下のようになる

![neural_network_layer](./img/neural_network_layer.png)

最後に、出力層に活性化関数 $σ()$ を導入すると、ニューラルネットワークの概念図が完成する

![neural_network_y](./img/neural_network_y.png)

In [ ]:
"""
3層ニューラルネットワークの実装
"""

# 内積関数 dot を使えるようにする
using LinearAlgebra

# ニューラルネットワーク構造体
mutable struct Network
    l::Int # ネットワーク層数
    b::Array{Float64,2} # バイアス: 2次元配列 {層番号i, 次層ニューロン番号n}
    w::Array{Float64,3} # 重み: 3次元配列 {層番号i, 次層ニューロン番号n, 前層ニューロン番号p}
end

# ニューラルネットワーク計算関数
## Network構造体, 中間層の活性化関数, 出力層の活性化関数, 入力信号 -> 出力信号 y
forward(network::Network, h::((::Float64)::Float64), σ::((::Float64)::Float64), x::Array{Float64,1})::Array{Float64,1} = begin
    # 1層目の計算
    a = [dot(x, network.w[1][n]) .+ network.b[1][n] for n in 1 : length(network.w[1])]
    z = h.(a)
    # 2層目以降の計算
    for i in 2 : network.l-1
        a = [dot(z, network.w[i][n]) .+ network.b[i][n] for n in 1 : length(network.w[i])]
        z = h.(a)
    end
    # 出力層の計算
    a = [dot(z, w[network.l][n]) .+ network.b[network.l][n] for n in 1 : length(network.w[network.l])]
    y = σ.(a)
end